In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns,SimpleMovingAverage
from quantopian.pipeline.classifiers.fundamentals import Sector  
from quantopian.pipeline.filters import QTradableStocksUS
from time import time

import alphalens as al 


In [ ]:
MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}


In [ ]:
class Zscore(CustomFactor):
    """
    """
    inputs = [USEquityPricing.close,]
    window_length = 20
    
    def compute(self, today, assets, out, close):
        out[:] = stats.zscore(close, axis=0)[-1] 
        
        

def my_sma():
    
    base_universe= QTradableStocksUS()
    
    sma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=60)
    zscore = Zscore()
    adj_close = USEquityPricing.close.latest 
    sector = Sector()

    myfactor = sma /adj_close #adj_close / sma 
    zscored_ratio = myfactor.zscore(mask=base_universe)
    
    return Pipeline(
        columns = {
            'zscore': zscore,
            'zscored_ratio':zscored_ratio, 
            'adj_close':adj_close,
            'sma':sma,
            'Sector' : Sector(),
            'myfactor':myfactor,
        },
        screen=zscored_ratio.notnull() 
    )


In [ ]:
start_timer = time()
results = run_pipeline(my_sma(), '2016-01-01', '2018-01-01')
end_timer = time()
results.fillna(value=0);


In [ ]:
print "Time to run pipeline %.2f secs" % (end_timer - start_timer)



In [ ]:
results.head()

In [ ]:
## 全期間で株式データを取得する。
asset_list = results.index.levels[1]
len(asset_list)

In [ ]:
prices = get_pricing(asset_list, start_date='2016-01-01', end_date='2018-01-01', fields='close_price')
prices.head()

In [ ]:
## alphalens のテストに使う収益率の日数。
periods = (1,5,10)
## factor をアルファレンズにかける。
## 
factor_data = al.utils.get_clean_factor_and_forward_returns(factor=results["myfactor"],
                                                           prices=prices,
                                                           groupby=results["Sector"],
                                                           groupby_labels=MORNINGSTAR_SECTOR_CODES,
                                                           periods=periods,
                                                           quantiles=10)

In [ ]:
## factor_quantileが、Factorでランキングされたあとに分位されて、グループ化された。
factor_data.head()

In [ ]:
len(factor_data.loc["2016-01-04"][factor_data.loc["2016-01-04"]["factor_quantile"]==5])


In [ ]:
al.tears.create_returns_tear_sheet(factor_data, by_group=True)

In [ ]:
al.tears.create_information_tear_sheet(factor_data, by_group=True,)
